In [1]:
import pandas as pd

In [19]:
import httpx
from selectolax.parser import HTMLParser
import re
import pandas as pd

base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'

res = httpx.get(base_url+event_url)

html = HTMLParser(res.text)

matches = html.css('a.wf-module-item')

completed_matches = [match for match in matches if match.css_first('div.ml-status').text() == 'Completed']

for match in completed_matches[:2]:
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '

    print(match_sum:=match_result.join(match_name))

    tables = match_html.css('table.wf-table-inset')

    for table in tables:
        for t in pd.read_html(table.html):
            t.iloc[:,2:] = t.iloc[:,2:].applymap(lambda r :  r.split(' ')[0])
            t = t.dropna(axis='columns',how='all')
            display(t)

KOI 0:2 NRG Esports


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,trexx KOI,1.24,304,27,/,6,+11,79%,206,35%,2,1,+1
1,sheydos KOI,1.15,193,17,/,4,+5,71%,124,28%,1,2,-1
2,Wolfen KOI,0.81,168,14,/,3,-4,67%,113,28%,5,8,-3
3,starxo KOI,0.73,134,10,/,11,-8,50%,92,23%,0,4,-4
4,koldamenta KOI,0.59,73,5,/,12,-14,54%,55,16%,0,1,-1


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,s0m NRG,1.42,295,24,/,4,+11,79%,184,29%,4,0,+4
1,crashies NRG,1.27,244,22,/,6,+7,71%,139,25%,3,2,+1
2,Victor NRG,1.00,161,14,/,9,-1,71%,97,17%,2,3,-1
3,FNS NRG,1.00,165,12,/,3,-2,79%,115,17%,4,0,+4
4,ardiis NRG,0.94,156,11,/,9,-5,75%,122,21%,3,3,0


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,trexx KOI,1.24,275,46,/,9,+19,76%,181,32%,3,4,-1
1,sheydos KOI,0.95,158,27,/,6,+3,70%,107,24%,2,4,-2
2,starxo KOI,0.76,123,17,/,22,-12,70%,79,18%,0,6,-6
3,Wolfen KOI,0.75,187,28,/,7,-10,61%,123,33%,10,13,-3
4,koldamenta KOI,0.72,110,17,/,19,-17,63%,75,21%,1,3,-2


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,crashies NRG,1.26,220,36,/,17,+9,74%,139,27%,5,4,+1
1,s0m NRG,1.25,227,36,/,12,+10,83%,147,34%,4,4,0
2,FNS NRG,1.12,173,24,/,18,0,83%,110,17%,5,0,+5
3,ardiis NRG,1.08,190,27,/,12,-1,76%,128,18%,10,4,+6
4,Victor NRG,0.98,172,29,/,10,-1,70%,110,24%,6,4,+2


...

,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Cloud GIA,1.96,372,26,/,1,+17,94%,244,38%,3,1,+2
1,nukkye GIA,1.52,216,12,/,13,+4,94%,135,25%,1,2,-1
2,rhyme GIA,1.45,204,13,/,10,+5,82%,125,36%,1,1,0
3,hoody GIA,1.20,178,10,/,3,+2,94%,129,14%,2,1,+1
4,Fit1nho GIA,1.01,159,10,/,4,-2,88%,112,17%,3,2,+1


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,takej DFM,1.00,197,13,/,5,0,60%,120,42%,2,1,+1
1,Reita DFM,0.79,208,25,/,8,-5,57%,137,44%,3,8,-5
2,Suggest DFM,0.72,155,21,/,14,-10,65%,91,30%,4,5,-1
3,Anthem DFM,0.71,159,19,/,6,-10,68%,109,32%,3,2,+1
4,xnfri DFM,0.66,151,19,/,6,-11,57%,107,19%,1,2,-1
5,Seoldam DFM,0.27,129,7,/,2,-9,53%,80,31%,2,4,-2


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Cloud GIA,1.58,287,42,/,5,+21,89%,191,39%,4,1,+3
1,nukkye GIA,1.41,233,32,/,17,+14,86%,151,33%,5,4,+1
2,rhyme GIA,1.32,201,28,/,19,+7,81%,132,36%,3,4,-1
3,hoody GIA,1.25,208,27,/,6,+7,86%,159,23%,5,2,+3
4,Fit1nho GIA,0.97,161,20,/,8,-4,76%,111,21%,5,4,+1


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,takej DFM,1.00,197,13,/,5,0,60%,120,42%,2,1,+1
1,Anthem DFM,0.95,190,13,/,3,-2,65%,121,33%,3,1,+2
2,Reita DFM,0.77,152,10,/,6,-5,55%,105,45%,1,5,-4
3,xnfri DFM,0.74,200,12,/,3,-5,65%,136,16%,1,1,0
4,Suggest DFM,0.69,159,11,/,8,-7,55%,92,22%,1,4,-3


,Unnamed: 0,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,nukkye GIA,1.30,249,20,/,4,+10,80%,165,39%,4,2,+2
1,hoody GIA,1.30,238,17,/,3,+5,80%,185,36%,3,1,+2
2,rhyme GIA,1.20,198,15,/,9,+2,80%,137,37%,2,3,-1
3,Cloud GIA,1.19,202,16,/,4,+4,85%,146,41%,1,0,+1
4,Fit1nho GIA,0.93,162,10,/,4,-2,65%,110,24%,2,2,0
